# Every Data of this notebook can be downloaded from : 

### Url : https://www.dropbox.com/sh/aj110jc65wrzm5g/AAAC_A76q7THYzZ2w5p50GQ3a?dl=0

# Reference 

### Url : http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/

# Load Packages

In [ ]:
!pip install SimpleITK SimpleITK
!pip install numpy

import SimpleITK as sitk
import numpy as np
import os
import matplotlib.pyplot as plt

# Generate Images

In [ ]:
image_2D = sitk.Image(64, 64, sitk.sitkFloat32)

In [ ]:
print(image_2D)

In [ ]:
image_2D_npy = sitk.GetArrayFromImage(image_2D)
print(image_2D_npy)
plt.imshow(image_2D_npy)
plt.show()

# Image Processing Exercise - with Kaggle pneumothorax dataset

## Sign-in and download pneumothorax data

### Url : https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation/data

### <font color='red'>Download Data to current directory!!!</font>

In [ ]:
# Load files

path = os.path.join('.', 'stage_2_images')
print(path)
filelist = os.listdir(path)

In [ ]:
# Print first file of files

print(filelist[0])

In [ ]:
# Try load first file with sitk

dcm = sitk.ReadImage(filelist[0])

In [ ]:
# Try load first file with path appended

filelist = [os.path.join(path, i) for i in filelist]
print(filelist)

In [ ]:
k = np.random.choice(len(filelist), 1, replace=False)[0]

dcm = sitk.ReadImage(filelist[k])

print(dcm)

In [ ]:
# Show dicom pixel values

npy = sitk.GetArrayFromImage(dcm)
plt.imshow(npy)
plt.show()

In [ ]:
# print shape of image

print(npy.shape)

In [ ]:
# Resize image shape

npy = npy.reshape((1024,1024)) # same as npy = npy.squeeze()

# Show numpy array
plt.imshow(npy)
plt.show()

In [ ]:
# Show image with gray colormap

plt.imshow(npy, cmap = plt.cm.gray)
plt.show()

In [ ]:
# Show metadata of dcm file

for k in dcm.GetMetaDataKeys():
    v = dcm.GetMetaData(k)
    print("({0}) == \"{1}\"".format(k,v))

In [ ]:
# Show pixel bit type
print(type(npy))
print(type(npy[0][0]))

In [ ]:
# binary representation of 1024

binary1024 = bin(1024) # bin(ary) representation of 1024
print(binary1024)
print(type(binary1024)) # data type of binary1024

In [ ]:
# Representation of -1024 with two's complement

a = np.binary_repr(-1024, width=16)
print(a)

In [ ]:
# 2의 보수(Two's Complement)
#  1024 = 0000001000000000
# -1024 = 1111110000000000
#
#
# 10000000000000000 <- 17-bit complementary number
# -0000001000000000 <- 1024
# =================
#  1111111000000000 <- Two's Complement of 1024( = -1024)


def twos_comp(binary_string, bits):
    val = int(binary_string, 2)
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0:
        val = val - (1 << bits)
    return val

print(twos_comp(a, 16))

# Image Processing Exercise - with TCGA dataset

## Using CT Image with 12, 16, 24 bit Dicom files

### Url : https://wiki.cancerimagingarchive.net/display/Public/TCGA-LUAD

### <font color='red'>Download Data to current directory!!!</font>

In [ ]:
# Get data

CT_directory = os.path.join('.', 'siim-medical-image-analysis-tutorial', 'dicom_dir')

CT_list = os.listdir(CT_directory)

CT_list = [os.path.join(CT_directory, i) for i in CT_list]

# Read random data from data list

k = np.random.choice(len(CT_list), 1, replace=False)[0]

CT = CT_list[k]

In [ ]:
# Read randomly chosen data

CT_dcm = sitk.ReadImage(CT)

In [ ]:
# Get numpy array of chosen data

CT_npy = sitk.GetArrayFromImage(CT_dcm)

In [ ]:
# Print Max pixel value of chosen dicom
print(np.max(CT_npy))

# Print shape of chosen dicom
print(CT_npy.shape)

# Print bit type of pixel value of chosen dicom
print(type(CT_npy[0][0][0]))
print(CT_dcm.GetPixelIDTypeAsString())

In [ ]:
plt.imshow(CT_npy.reshape((512,512)), cmap = plt.cm.gray)
plt.show()

In [ ]:
for k in CT_dcm.GetMetaDataKeys():
    v = CT_dcm.GetMetaData(k)
    print("({0}) == \"{1}\"".format(k,v))


### Dicom Tags : https://www.dicomlibrary.com/dicom/dicom-tags/

In [ ]:
print(CT_npy) # Signed dicom file

In [ ]:
print("Max pixel value : ", CT_npy.max())
print("Min pixel value : ", np.min(CT_npy))

In [ ]:
# Normalization for dicom
# 1. Standardization : mean to be 0
standardization = (CT_npy - np.mean(CT_npy)) / np.std(CT_npy)

# 2. MinMax scaling 1 : 0~1
minmax1 = (CT_npy - np.min(CT_npy)) / (np.max(CT_npy) - np.min(CT_npy))

# 3. MinMax scaling 2 : -1~1
minmax2 = 2*(CT_npy - np.min(CT_npy)) / (np.max(CT_npy) - np.min(CT_npy)) - 1

In [ ]:
plt.hist(CT_npy.flatten(), bins=50)
plt.title("Original histogram")
plt.show()

In [ ]:
plt.hist(standardization.flatten(), bins=50)
plt.title("Standardization")
plt.show()

In [ ]:
plt.hist(minmax1.flatten(), bins=50)
plt.title("MinMax1")
plt.show()

In [ ]:
plt.hist(minmax2.flatten(), bins=50)
plt.title("MinMax2")
plt.show()

## Transformations and Resampling

In [ ]:
# How to transform image - 1. linear interpolation



rate = 2


reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size
print("Original voxel size : ", voxel_size)
plt.imshow(sitk.GetArrayFromImage(reference_img).squeeze(), cmap = 'gray')
plt.show()

resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkLinear)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
resamplefilter.SetOutputSpacing(voxel_size)
a, b, c = reference_img.GetSize()
print("Original image size : ({0}, {1}, {2})".format(a,b,c))

a, b= a//rate, b//rate
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))

### Why image has been sliced?

In [ ]:
# How to transform image - 1. linear interpolation


rate = 2


reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size
print("Original voxel size : ", voxel_size)
plt.imshow(sitk.GetArrayFromImage(reference_img).squeeze(), cmap = 'gray')
plt.show()
resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkLinear)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing(voxel_size) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()
print("Original image size : ({0}, {1}, {2})".format(a,b,c))

a, b= a//rate, b//rate
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))

In [ ]:
# How to transform image - 2. nearest neighborhood method


rate = 0.5


reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size
print("Original voxel size : ", voxel_size)
plt.imshow(sitk.GetArrayFromImage(reference_img).squeeze(), cmap = 'gray')
plt.show()
resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkNearestNeighbor)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing([1.0,1.0,1.0]) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()
print("Original image size : ({0}, {1}, {2})".format(a,b,c))

a, b= int(a/rate), int(b/rate)
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))

In [ ]:
# Comparison of interpolators



# 1. Linear interpolator

rate = 0.5

reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size
resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkLinear)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing([1.0,1.0,1.0]) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()

a, b= int(a/rate), int(b/rate)
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.figure(figsize=(15,15))
plt.title("1. Linear interpolation")
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))


# 2. Nearest neighborhood interpolation


reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size

resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkNearestNeighbor)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing([1.0,1.0,1.0]) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()

a, b= int(a/rate), int(b/rate)
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.figure(figsize=(15,15))
plt.title("2. Nearest neighborhood interpolation")
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))


# 3. B-spline interpolation


reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size

resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkBSpline)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing([1.0,1.0,1.0]) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()

a, b= int(a/rate), int(b/rate)
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.figure(figsize=(15,15))
plt.title("3. B-spline interpolation")
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))


# 4. Gaussian interpolation

reference_img = CT_dcm
voxel_size = CT_dcm.GetSpacing() # Get Image voxel size

resamplefilter = sitk.ResampleImageFilter()
resamplefilter.SetInterpolator(sitk.sitkGaussian)
resamplefilter.SetOutputDirection(reference_img.GetDirection())
resamplefilter.SetOutputOrigin(reference_img.GetOrigin())
# resamplefilter.SetOutputSpacing([1.0,1.0,1.0]) Change this line to
x, y, z = reference_img.GetSpacing()
resamplefilter.SetOutputSpacing((x*rate, y*rate, z))
####################################################################
a, b, c = reference_img.GetSize()

a, b= int(a/rate), int(b/rate)
resamplefilter.SetSize((a,b,c))
newimage1 = resamplefilter.Execute(reference_img)
newimage1_npy = sitk.GetArrayFromImage(newimage1)
plt.figure(figsize=(15,15))
plt.title('4. Gaussian interpolation')
plt.imshow(newimage1_npy.squeeze(), cmap = 'gray')
plt.show()
print("New image size : ({0}, {1}, {2})".format(a,b,c))

# Introduction to Segmentation

In [ ]:
def img_show(npy_arr, figsize = (5,5), figtitle = None, colormap = 'gray', axis_onoff = 'off'):
    
    npy_arr = npy_arr.squeeze()
    
    assert len(npy_arr.shape) in [2,3,4]
    
    plt.figure(figsize = figsize)
    plt.imshow(npy_arr, cmap = colormap)
    plt.axis(axis_onoff)
    if figtitle != None:
        plt.title(figtitle)
    plt.show()

In [ ]:
# Randomly select CT image from TCGA dataset again

k = np.random.choice(len(CT_list), 1, replace = False)[0]

print(CT_list[k])
CT_dcm = sitk.ReadImage(CT_list[k])

img_show(sitk.GetArrayFromImage(CT_dcm))

In [ ]:
img_show(sitk.GetArrayFromImage(CT_dcm), figtitle = 'Original')

threshold = 1000
threshold_npy = CT_dcm>threshold
CT_255 = sitk.Cast(sitk.RescaleIntensity(CT_dcm), sitk.sitkUInt8)
overlaid1 = sitk.LabelOverlay(CT_255, threshold_npy)
img_show(sitk.GetArrayFromImage(overlaid1), figtitle = 'Threshold 1000')


threshold = 500
threshold_npy = CT_dcm>threshold
CT_255 = sitk.Cast(sitk.RescaleIntensity(CT_dcm), sitk.sitkUInt8)
overlaid2 = sitk.LabelOverlay(CT_255, threshold_npy)
img_show(sitk.GetArrayFromImage(overlaid2), figtitle = 'Threshold 500')


threshold = 100
threshold_npy = CT_dcm>threshold
CT_255 = sitk.Cast(sitk.RescaleIntensity(CT_dcm), sitk.sitkUInt8)
overlaid3 = sitk.LabelOverlay(CT_255, threshold_npy)
img_show(sitk.GetArrayFromImage(overlaid3), figtitle = 'Threshold 100')

### Save Image and Reload

In [ ]:
sitk.WriteImage(overlaid3, 'Segmentation_with_threshold_100.png', True)

In [ ]:
png = sitk.GetArrayFromImage(sitk.ReadImage('Segmentation_with_threshold_100.png'))
plt.imshow(png)
plt.axis('off')
plt.show()

# Transformations

In [ ]:
a = np.random.randint(0,100, 2)
print("Coordinate of random vector : ", a)

def transformation_plot(points, xrange = [0,150], yrange = [0,150], dottype = 'o'):
    colors = [i + dottype for i in ['g', 'r', 'b', 'c', 'm', 'y', 'k', 'w']]
    if len(points)>len(colors):
        print("Number of points should be smaller or less than {0}".format(len(colors)))
    assert len(points)<=len(colors)
    
    for i in range(len(points)):
        assert len(points[i]) == 2
    plt.xlim(xrange[0], xrange[1])
    plt.ylim(yrange[0], yrange[1])
    
    for j in range(len(points)):
        plt.plot(points[j][0],points[j][1], colors[j])
    plt.show()
transformation_plot([a])

### Translation transform

$$ y = x + b$$

#### where $b$ is translation vector

In [ ]:
dimension = 2
move = [10,15] # Same as Offset

translation = sitk.TranslationTransform(dimension, move)
print(translation)

In [ ]:
translated = translation.TransformPoint(a.tolist())
print("Original point : ({0}, {1})".format(a[0], a[1]))
print("Translated point : ({0}, {1})".format(translated[0], translated[1]))
transformation_plot([a, translated])

### Euler2DTransform

$$ y = R_{\theta}x + b$$

#### where $R_{\theta}$ is rotation matrix and $b$ is translation vector

In [ ]:
p1 = np.random.randint(0, 100, 2)
p2 = np.random.randint(0, 100, 2)
p3 = np.random.randint(0, 100, 2)
p1, p2, p3 = p1.tolist(), p2.tolist(), p3.tolist()

origin = [0,0]

print("Point 1 : ({0}, {1})".format(p1[0], p1[1]))
print("Point 2 : ({0}, {1})".format(p2[0], p2[1]))
print("Point 3 : ({0}, {1})".format(p3[0], p3[1]))
transformation_plot([p1, p2, p3, origin], xrange=[-150,150], yrange=[-150,150])

In [ ]:
move = [10,15]

Euler2D = sitk.Euler2DTransform()
Euler2D.SetTranslation(move)
Euler2D.SetAngle(np.pi/2)
transformed1 = Euler2D.TransformPoint(p1)
transformed2 = Euler2D.TransformPoint(p2)
transformed3 = Euler2D.TransformPoint(p3)

print("Transformed 1 : ({0}, {1})".format(transformed1[0], transformed1[1]))
print("Transformed 2 : ({0}, {1})".format(transformed2[0], transformed2[1]))
print("Transformed 3 : ({0}, {1})".format(transformed3[0], transformed3[1]))
transformation_plot([transformed1, transformed2, transformed3, origin], xrange=[-150,150], yrange=[-150,150])

#### Types of transformations

1. sitk.TranslationTransform
2. sitk.VersorTransform
3. sitk.VersorRigid3DTransform
4. sitk.Euler2DTransform
5. sitk.Euler3DTransform
6. sitk.Similarity2DTransform
7. sitk.Similarity3DTransform
8. sitk.ScaleTransform
9. sitk.ScaleVersor3DTransform
10. sitk.ScaleSkewVersor3DTransform
11. sitk.AffineTransform
12. sitk.BSplineTransform
13. sitk.DisplacementFieldTransform
14. sitk.Transform

# Histogram Equalization

In [ ]:
k = np.random.choice(len(filelist), 1, replace=False)[0]

X_dcm = sitk.ReadImage(filelist[k])

X_npy = sitk.GetArrayFromImage(X_dcm)

plt.imshow(X_npy.squeeze(), cmap = plt.cm.gray)
plt.show()
plt.hist(X_npy.flatten(), bins = 50)
plt.show()

In [ ]:
histogramequalization = sitk.AdaptiveHistogramEqualizationImageFilter()

In [ ]:
alpha = 1
beta = 1

reference_img = CT_npy
histogramequalization.SetAlpha(alpha)
histogramequalization.SetBeta(beta)
equalized = histogramequalization.Execute(X_dcm)
equalized_npy = sitk.GetArrayFromImage(equalized).squeeze()
plt.imshow(equalized_npy, cmap = plt.cm.gray)
plt.show()
plt.hist(equalized_npy.flatten(), bins = 50)
plt.show()

In [ ]:
alpha = 0
beta = 0

reference_img = CT_npy
histogramequalization.SetAlpha(alpha)
histogramequalization.SetBeta(beta)
equalized = histogramequalization.Execute(X_dcm)
equalized_npy = sitk.GetArrayFromImage(equalized).squeeze()
plt.imshow(equalized_npy, cmap = plt.cm.gray)
plt.show()
plt.hist(equalized_npy.flatten(), bins = 50)
plt.show()

In [ ]:
alpha = 0
beta = 1

reference_img = CT_npy
histogramequalization.SetAlpha(alpha)
histogramequalization.SetBeta(beta)
equalized = histogramequalization.Execute(X_dcm)
equalized_npy = sitk.GetArrayFromImage(equalized).squeeze()
plt.imshow(equalized_npy, cmap = plt.cm.gray)
plt.show()
plt.hist(equalized_npy.flatten(), bins = 50)
plt.show()

In [ ]:
alpha = 1
beta = 0

reference_img = CT_npy
histogramequalization.SetAlpha(alpha)
histogramequalization.SetBeta(beta)
equalized = histogramequalization.Execute(X_dcm)
equalized_npy = sitk.GetArrayFromImage(equalized).squeeze()
plt.imshow(equalized_npy, cmap = plt.cm.gray)
plt.show()
plt.hist(equalized_npy.flatten(), bins = 50)
plt.show()

In [ ]:
maxi = 5
maxj = 5
for i in range(maxi+1):
    for j in range(maxj+1):
        alpha = i/maxi
        beta = j/maxj

        reference_img = CT_npy
        histogramequalization.SetAlpha(alpha)
        histogramequalization.SetBeta(beta)
        equalized = histogramequalization.Execute(X_dcm)
        equalized_npy = sitk.GetArrayFromImage(equalized).squeeze()
        plt.imshow(equalized_npy, cmap = plt.cm.gray)
        plt.title("Alpha : "+str(alpha) + ", Beta : "+str(beta))
        plt.show()
        plt.hist(equalized_npy.flatten(), bins = 50)
        plt.show()